In [28]:
import ee
import geemap
import pandas as pd

In [29]:
Map = geemap.Map()

In [30]:
df = pd.read_excel("SUBC.xlsx")
comunas = df["COD_SUBC"].unique().tolist()

In [31]:
def registros(cod, cant):
    diccionario = {}
    diccionario["SUBC"] = cod
    diccionario[year] = cant

    return diccionario

In [32]:
startDate = ['2001-01-01', '2002-01-01', '2003-01-01', '2004-01-01', '2005-01-01', '2006-01-01', '2007-01-01', '2008-01-01', '2009-01-01', '2010-01-01']
endDate = ['2002-01-01', '2003-01-01', '2004-01-01', '2005-01-01', '2006-01-01', '2007-01-01', '2008-01-01', '2009-01-01', '2010-01-01', '2011-01-01']

In [34]:
for i in zip(startDate,endDate):
    
    Fecha_inicial = str(i[0])
    Fecha_final = str(i[1])
    
    year = Fecha_inicial[:4]
    print(year)
    
    puntos = []

    for i in comunas:
        studyArea = ee.FeatureCollection("users/testHector/Subsubcuencas_v2")
        studyArea = studyArea.filterMetadata("COD_SUBC","equals", str(i).zfill(4))

        FIRMS_colection2 = ee.ImageCollection('FIRMS')
        FIRMS_colection = ee.ImageCollection('FIRMS')

        FIRMS4 =FIRMS_colection2 \
          .select(['T21']) \
          .filterDate(Fecha_inicial,Fecha_final) \
          .filterBounds(studyArea)

        FIRMS =FIRMS_colection \
          .select(['T21']) \
          .filterDate(Fecha_inicial,Fecha_final) \
          .filterBounds(studyArea)

        FIRMScount4  = ee.Image(FIRMS4.count()).clip(studyArea)
        FIRMSbinary4 = FIRMScount4.eq(FIRMScount4).rename('FIRMS_binary_alert_3')

        project_crs   = ee.Image(FIRMS.first()).projection().crs()
        scale = ee.Image(FIRMS.first()).projection().nominalScale()

        FIRMSpoint4 = FIRMSbinary4.reduceToVectors(
          geometry = studyArea,
          eightConnected=True,
          labelProperty='modis_fire',
          maxPixels=1e16,
          crs=project_crs,
          scale=scale,
          geometryType= 'centroid',
          bestEffort= True,
          tileScale= 16
        )

        numero_PI = ee.FeatureCollection(FIRMSpoint4).filterBounds(studyArea)

        cantidad_PI = numero_PI.size()
        cantidad =  cantidad_PI.getInfo()

        diccionario = registros(i, cantidad)
        puntos.append(diccionario.copy())
        
        
    data = pd.DataFrame(puntos)
    data.to_excel("subc_puntos_calor_anual/" + str(year) + ".xlsx", index=False)

2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
